In [1]:
import scipy.io
import pylab
import numpy as np
import datetime 
import ephem
import random
from scipy import spatial
import matplotlib.pyplot as plt
%matplotlib 
from astropy.table import Table
from astropy import units as unit
from astropy.coordinates import SkyCoord
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages
import astropy
import astropy.constants as const
from astropy.time import Time
from astropy.coordinates import EarthLocation
import sunpy.coordinates
from datetime import datetime, timezone
from math import atan, atan2, degrees, floor, pi, radians, sin, sqrt
from skyfield.api import earth, JulianDate, now, sun
data=scipy.io.loadmat('BNS-sources-trial-10saleem.mat')['data'] 
#sourceparameters
DL=data[:,0]
theta=data[:,1]
phi=data[:,2]

Using matplotlib backend: MacOSX


In [2]:
num = 220000
radius = const.R_earth.to(unit.m)  #Radius of earth in m
earth_rad = radius.value
lat = np.rad2deg(np.pi/2 - theta)
long = np.rad2deg(phi)


In [3]:
distance = DL*unit.Mpc

In [4]:
#snr at each detector
snrH=data[:,5]
snrL=data[:,6]
snrV=data[:,7]
snrK=data[:,8]
snrI=data[:,9]
#or for networks of 5 and 3 detectors, one can use directly,
snrLHVKI=np.sqrt(data[:,5]**2 + data[:,6]**2 + data[:,7]**2 +data[:,8]**2 + data[:,9]**2)  
# SNR at LHVKI
snrLHV=np.sqrt(data[:,5]**2 + data[:,6]**2 + data[:,7]**2 )  # LHV*
snrLHVK = np.sqrt(data[:,5]**2 + data[:,6]**2 + data[:,7]**2 + data[:,8]**2)
snrLHVI = np.sqrt(data[:,5]**2 + data[:,6]**2 + data[:,7]**2 + data[:,9]**2)
snrH, snrL, snrLHV,snrLHVK, snrLHVI

(array([1.50828927, 1.3218442 , 4.88369119, ..., 0.14711676, 1.92059118,
        3.73704228]),
 array([2.05597716, 0.7623997 , 3.94531224, ..., 0.87258219, 1.87887723,
        3.17437839]),
 array([2.60109925, 2.01527692, 6.56372816, ..., 2.09722166, 3.03471228,
        5.1190831 ]),
 array([2.9615534 , 2.05214273, 7.23216427, ..., 2.10021926, 3.04157166,
        5.57032074]),
 array([2.80024108, 2.33763304, 7.55768521, ..., 2.6656951 , 3.20978535,
        5.91029335]))

In [5]:
def radectolatlong(Ra,Dec,time): #input in degrees
    
    """
    Take the RA, Dec of GW trigger from the table and convert them to  longitude, latitude
    RA, Dec should be floating point numbers (not astropy.coordinates)
    time should be a astropy.Time.time object
    Output is in floating point numbers (not astropy.quantity or astropy.coordinates)
    """
    ra = np.deg2rad(Ra)
    dec = np.deg2rad(Dec)
    time_sid = (time.sidereal_time('apparent','greenwich')).rad
    lat = dec
    long = ra - time_sid 
    return lat,long #output in radians

def latlongtoradec(Lat,Long,time): #input in degrees
    """
    Take the Lat, Long of GW trigger from the saleem's table and convert them to  RA and DEC.
    """
    Lat = np.deg2rad(Lat)
    Long = np.deg2rad(Long)
    time_sid = (time.sidereal_time('apparent','greenwich')).rad
    
    DEC = Lat
    RA = Long + time_sid
    ra = np.rad2deg(RA) % 360  
    dec = np.rad2deg(DEC)
    return ra,dec #output in degree

def is_saa(lat, longi):
    """Return whether the satellite is in South Atlantic Anamoly or not.
    Hard bounds used from the data of RXTE and assumed that the SAA region is rectangular below the inclination of RXTE.
    
    ra - right ascention of the body in degrees
    dec - declination of the body in degrees
    time - UTC time in datetime format
    """
    # lat, longi = radec2latlong(ra, dec, time)
    #lat, longi = eci_ecef(ra, dec, time)
    lat = np.rad2deg(lat)
    longi = np.rad2deg(longi)
    if -23< lat <= -7 and -90<=longi<=-10:
        return True
    
#     if -90<=longi<=-10:
#         y = 7*longi/80 + 23/8
#         if lat < y:
#             return True
    if -100<=longi<=-35:
        y=14*longi/65+189/13
        if lat<=y:
            return True
    
    if -35<longi<-10:
        y=-14/25*longi-63/5
        if lat<y:
            return True
    
    
    if -10<=longi<=40:
        z = -9*longi/25 -43/5
        if lat<z: 
            return True
	

    if -50<= lat <=-23 and -90<=longi<=40:
        return True
    else: return False 

def getxyz(theta,phi):
    """
    Convert cartesian coordinates into theta phi for ANY FRAME OF REFERENCE
    Theta ranges from 0 to pi i.e angle from z-axis axis, and is a floating point number in RADIANS
    Phi ranges from 0 to 2*pi, and is a floating point number in RADIANS
    x,y z are floating point numbers - and make a unit vector
    """
    theta = np.pi/2 - theta
    x=np.sin(theta)*np.cos(phi)
    y=np.sin(theta)*np.sin(phi)
    z=np.cos(theta)
    return x,y,z
def getnorm(vec):
    """
    Takes in the vector and get its norm and returns unit vector
    """
    norm = np.linalg.norm(vec)
    #print (norm)
    return vec/norm

In [6]:
df = pd.read_csv('/Users/jinaypatel/Desktop/Daksha/Events(triggers).csv')
t1 = df.sort_values(by=['Time in isot'])
t2 = t1.reset_index(drop=True)   #resetting the index 
ra = t2['RA in degree']   #in degree
dec = t2['DEC in degree']
ht = t2['Time in jd']
homotime = Time(ht,format='jd')
source_ra = np.empty(num,dtype=float)
source_dec = np.empty(num,dtype=float)

for i in range(num):
    source_ra[i] = ra[i]
    source_dec[i] = dec[i]
    
"""
source_ra and source_dec are for removing the unnecessary index in ra and dec.
"""

'\nsource_ra and source_dec are for removing the unnecessary index in ra and dec.\n'

In [7]:
#time1 = Time('2022-01-01T00:00:00') 
#time2 = Time('2023-01-01T00:00:00')
#homotime = Time(np.random.uniform(low=time1.jd, high=time2.jd, size=len(theta)), format="jd").sort()
#"""
#homotime gives random but uniform sorted time in jd format.
#"""
"""Converting homotime to iso format."""
time_iso = homotime.iso

In [8]:
sunloc = astropy.coordinates.get_sun(homotime)   # Gives the sun's location 

In [9]:
sunRA = sunloc.ra.deg  #sun's RA in earth's frame in degrees
sunDEC = sunloc.dec.deg #sun's DEC in earth's frame in degrees
sunDIST = sunloc.distance.AU*1.496e11 #sun distance from earth in m

In [10]:
"""
Given below is the TLE format data for the satellite, i.e Astrosat. 
sublat and sublong are the latitude and longitude of the satellite at any instant.
alt1 and alt2 are altitude of sat above the earth's surface.
"""
line0 = 'ASTROSAT' #Daksha
line1 = '1 40930U 15052A   22302.17850942  .00000772  00000-0  19439-4 0  9990'
line2 = '2 40930   5.9975   0.5159 0009292 339.1937  20.7998 14.76165238112787'
line3 = '2 40930   5.9975   0.5159 0009292 339.1937 200.7998 14.76165238112787'
sublat1 = np.empty(num,dtype=np.float)  
sublong1 = np.empty(num,dtype=np.float)  
sublat2 = np.empty(num,dtype=np.float)  
sublong2 = np.empty(num,dtype=np.float)

"""earth_ra and earth_dec are ra and dec of earth in refrence frame of satellite."""

earth_ra = np.empty(num,dtype=np.float)
earth_dec = np.empty(num,dtype=np.float)
elevation = np.empty(num,dtype=np.float)  # its elevation of 1st daksha satellite

# line0 = 'SWIFT'#BAT
# line1 = '1 28485U 04047A   22302.18405080  .00001301  00000-0  60140-4 0  9997'
# line2 = '2 28485  20.5572 277.8742 0011565 130.4507 229.6936 15.03911323708409'
# line0 = 'FGRST (GLAST)'#Fermi
# line1 = '1 33053U 08029A   22100.45970249  .00000511  00000-0  12538-4 0  9998'
# line2 = '2 33053  25.5819 140.4060 0012690 120.1881 239.9912 15.11069703542313'
astrosat1 = ephem.readtle(line0,line1,line2)
astrosat2 = ephem.readtle(line0,line1,line3)

for i in range(num):
    time = time_iso[i]
    astrosat1.compute(time)
    astrosat2.compute(time)
    sublat1[i] = (astrosat1.sublat)          #sublat in radians
    sublong1[i] = (astrosat1.sublong)        #sublong in radian
    sublat2[i] = (astrosat2.sublat)          #sublat in radians
    sublong2[i] = (astrosat2.sublong)
    earth_ra[i] = (np.rad2deg(astrosat1.g_ra) + 180 ) % 360
    earth_dec[i] = - np.rad2deg(astrosat1.g_dec)
    elevation[i] = astrosat1.elevation

In [11]:
"""Satellite RA and DEC"""
sat_ra = (sunRA+180) % 360  #RA is in degrees
sat_dec = -sunDEC   #DEC is in degrees

In [12]:
"""
Distance is 1 m as we can get unit vector.
c and event are the SkyCoord object for the satellite and the sources, respectively.
sv is the vector from sun to earth/satellite in ICRS frame. 
Similarly, trig is the vector from earth/satellite to source.
"""
c = SkyCoord(ra=sat_ra*unit.degree, dec=sat_dec*unit.degree, distance=1*unit.m,frame='gcrs')
sv = np.transpose(np.array([c.cartesian.x.value, c.cartesian.y.value, c.cartesian.z.value]))

event = SkyCoord(ra=source_ra*unit.degree,dec=source_dec*unit.degree,distance=1*unit.m,frame='gcrs')
trig = np.transpose(np.array([event.cartesian.x.value, event.cartesian.y.value, event.cartesian.z.value]))

In [13]:
"""
Define random luminosity of sources b/w 10^-7 and 10^-6. """
lumino = 10**np.random.uniform(-7, -6, num) * (DL / 200.)**2


In [14]:
def is_visible(homotime, trig_ra, trig_dec, satephem):
    """
    Check for the visibility of source, i.e whethear the source is block by earth or not.
    eclipse angle is the minimum a1ngle below which the source(trigger) will not be seen from satellite frame.
    earth_ra and earth_dec are ra and dec of earth in refrence frame of satellite.
    trig_ra and trig_dec are ra and dec of trigger.
    And the ra and dec of trig will be same in both frames, i.e in satellite and earth as they are too far away.
    sep is the angular distance between trig and esrth in satellite frame.
    satephem.g_ra and satephem.g_dec return angle in radians.
    """
    time = homotime.iso
    satephem.compute(time)
    eclipse_angle = np.arcsin(const.R_earth.to(unit.m).value/(const.R_earth.to(unit.m).value+satephem.elevation))*unit.rad
    earth_ra = (np.rad2deg(satephem.g_ra) + 180 ) % 360
    earth_dec = - np.rad2deg(satephem.g_dec)
    c1 = SkyCoord(earth_ra*unit.deg,earth_dec*unit.deg,frame='gcrs')
    c2 = SkyCoord(trig_ra*unit.deg,trig_dec*unit.deg,frame='gcrs')
    sep = c1.separation(c2)
    if sep.value > eclipse_angle.to(unit.deg).value:    # to() function is used here to convert rad to deg.
        return True
    else:  
        return False

In [15]:
min(source_dec)

-89.81381370704906

In [16]:
def transform_antisun(astrosat):   
    """Transforming coordinate system to satellite's frame for anti-sun orbit. 
zaxis is the axis from sun to earth and is equal to sv which unit vector.
satvecold is the unit vector from earth to satellite. This unit vector when intersect with satellite, 
the coordinates of that point will be the distance from center of earth times the unit vector.
(satx,saty,satz) are coordinate of the above described point. Now the equation of plane which is perpendicular
to the sv vector is given by sv[0]*(x-satx) + sv[1](y-saty) + sv[2](z-satz) = 0. 
(This is a plane with normal vector sv and contain the point [satx,saty,satz] on it).
The intersection of this plane with the equatorial plane i.e z=0 gives the line sv[0]*(x-satx) + sv[1](y-saty) = 0.
i.e. sv[0]*x + sv[1]*y = sv[0]*satx + sv[1]*saty.

trigvec is the trigger unit vector. yprojtrig and xprojtrig are the the projection vector on x-y plane in satellite
coordinate frame and are used to determine the theta and phi of source.
theta is angle from z-axis and phi is angle in xy-plane from x-axis.
"""
    thetatransarray = np.empty(num,dtype=float)
    phitransarray = np.empty(num,dtype=float)
    for i in range(len(theta)):
        if i%(int((num)/10))==0:print(int(i*100/(num)), '% completed', end='\r')
        zaxis = sv[i]
        #xaxis = getnorm(np.cross(zaxis,satvecold))
        taninv = np.arctan(sv[i][0]/sv[i][1])
        xaxis = [-np.cos(taninv),np.sin(taninv),0]
        yaxis = getnorm(np.cross(zaxis,xaxis))
        trigvec = trig[i]

        thetatrans = np.arccos(np.dot(trigvec,zaxis))
        yprojtrig = np.linalg.norm(trigvec)*np.dot(trigvec,yaxis)
        xprojtrig = np.linalg.norm(trigvec)*np.dot(trigvec,xaxis)
        phitrans = np.arctan2(yprojtrig, xprojtrig)
        thetatransarray[i] = thetatrans
        phitransarray[i] = phitrans
    return thetatransarray,phitransarray

#theta1_antisun,phi1_antisun = transform_antisun(astrosat1)
#theta2_antisun,phi2_antisun = transform_antisun(astrosat2)

## Test cases using 3D plots

In [17]:
"""
to obtain the cartesian coordinate of earth in the satellite frame."""
"""
It gives the position of earth w.r.t satellite. Returns the position in cartesian coordinates."""
def earthpos_wrtsat(time,satephem):
    t = time.iso
    satephem.compute(t)
    earth_ra = (np.rad2deg(satephem.g_ra) + 180 ) % 360
    earth_dec = - np.rad2deg(satephem.g_dec)
    cart = SkyCoord(ra = earth_ra*unit.deg, dec = earth_dec*unit.deg,frame='gcrs')
    return cart.cartesian.x.value,cart.cartesian.y.value,cart.cartesian.z.value

In [18]:
"""
used for plotting the 3D plot."""
"""n is the nth source. d is used to find the cartesian coordinates of source w.r.t earth or satellite as the 
source is too far away."""
n=213467

taninv = np.arctan(sv[n][0]/sv[n][1])
satx = getnorm([-np.cos(taninv),np.sin(taninv),0])
satz = sv[n]
saty = getnorm(np.cross(satz,satx))
d = SkyCoord(ra=source_ra[n]*unit.degree,dec=source_dec[n]*unit.degree,distance=1*unit.m)
trigvec = [d.cartesian.x.value, d.cartesian.y.value, d.cartesian.z.value]
earthvec = earthpos_wrtsat(homotime[n],astrosat1)
is_visible(homotime[n], source_ra[n], source_dec[n], astrosat1)

True

In [51]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib 
import numpy as np
from itertools import product, combinations


fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_aspect("equal")

# draw cube
r = [-0.05, 0.05]
for s, e in combinations(np.array(list(product(r, r, r))), 2):
    if np.sum(np.abs(s-e)) == r[1]-r[0]:
        ax.plot3D(*zip(s, e), color="r")

#earth
u, v = np.mgrid[0:2*np.pi:200j, 0:np.pi:200j]
scale=0.75
x = scale*np.cos(u)*np.sin(v) + earthvec[0]
y = scale*np.sin(u)*np.sin(v) + earthvec[1]
z = scale*np.cos(v) + earthvec[2]
ax.plot_wireframe(x, y, z, color="b")


# draw a point
ax.scatter([0], [0], [0], color="g", s=100)
ax.scatter(1.5*trigvec[0],1.5*trigvec[1],1.5*trigvec[2],color='black',s=10)

# draw a vector
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d


class Arrow3D(FancyArrowPatch):

    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0, 0), (0, 0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        FancyArrowPatch.draw(self, renderer)
s1 = Arrow3D([0,2*trigvec[0]],[0,2*trigvec[1]],[0,2*trigvec[2]], mutation_scale=20,
            lw=1, arrowstyle="-|>", color="k")

e = Arrow3D([0,earthvec[0]],[0,earthvec[1]],[0,earthvec[2]], mutation_scale=20,
            lw=1, arrowstyle="-|>", color="k")
sun = Arrow3D([0,-2*satz[0]],[0,-2*satz[1]],[0,-2*satz[2]], mutation_scale=20,
            lw=1, arrowstyle="-|>", color="k")

ax.quiver(0,0,0, satx[0], satx[1], satx[2] , color='blue')
ax.quiver(0,0,0, saty[0], saty[1], saty[2], color='green')
ax.quiver(0,0,0, satz[0],satz[1],satz[2], color='red')
ax.text3D(satx[0], satx[1], satx[2] , 'X')
ax.text3D(saty[0], saty[1], saty[2], 'Y')
ax.text3D(satz[0],satz[1],satz[2], 'Z')
ax.text3D(1.5*trigvec[0], 1.5*trigvec[1], 1.5*trigvec[2] , 'Source')
ax.text3D(-2*satz[0],-2*satz[1],-2*satz[2], 'Sun')
ax.text3D(earthvec[0],earthvec[1],earthvec[2], 'Earth')
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_zlim(-1, 1)
ax.add_artist(e)
ax.add_artist(s1)
ax.add_artist(sun)
plt.show()


Using matplotlib backend: MacOSX


In [31]:
trigvec[0],trigvec[1],trigvec[2
                             ]

(-0.6378536082441154, 0.2766437173084801, 0.7187565847515474)

### Sensitivity Part

In [19]:
sensfile = np.loadtxt('/Users/jinaypatel/Desktop/Daksha/sensitivity_20det.csv', delimiter=',',skiprows=1)

In [21]:
"""
thetadeg1, thetadeg2, phideg1 and phideg2 are the angles in degree.
theta1 is from 0 to np.pi rad and phi1 is from 0 to 2*np.pi rad.
thetadeg1 is from 0 to 180 deg and phideg1 is from 0 to 360 deg."""

theta1,phi1 = transform_antisun(astrosat1)   
#theta2,phi2 = transform_antisun(astrosat2)
thetadeg1 = np.rad2deg(theta1) 
#thetadeg2 = np.rad2deg(theta2)
phideg1 = np.rad2deg(phi1 % 2*np.pi)
#phideg2 = np.rad2deg(phi2)
newphi = phi1 % np.pi

In [ ]:
"""sensarray = np.empty(len(sensfile),dtype=float)
sensitivitydata1 = np.empty(num,dtype=float)
#sensitivitydata2 = []
sensfilethetaphi = []
for i in range(len(sensfile)):
    sensfilethetaphi.append((sensfile[i][0], sensfile[i][1]))
    sensarray[i] = sensfile[i][3]"""

In [74]:
"""
sens_source1 is the SkyCoord class for the theta-phi calculated from the code and sens_sat is the SkyCoord class 
for previously calculated theta-phi from the projection areas.
nearest gives the index of the nearest theta-phi in the given theta-phi file."""

"""sens_source1 = SkyCoord(phideg1*unit.deg,(thetadeg1-90)*unit.deg,frame='gcrs')
#sens_source2 = SkyCoord(dec=(theta2-np.pi/2)*unit.rad,ra = phi2*unit.rad,distance=1*unit.m,frame='gcrs')
sens_sat = SkyCoord(np.transpose(sensfilethetaphi)[1]*unit.rad,(np.transpose(sensfilethetaphi)[0]-np.pi/2)*unit.rad,frame='gcrs')
nearest1 = astropy.coordinates.match_coordinates_sky(sens_source1, sens_sat, nthneighbor=1, storekdtree='kdtree_sky')[0]
#nearest2 = astropy.coordinates.match_coordinates_sky(sens_source2, sens_sat, nthneighbor=1, storekdtree='kdtree_sky')[0]"""

"sens_source1 = SkyCoord(phideg1*unit.deg,(thetadeg1-90)*unit.deg,frame='gcrs')\n#sens_source2 = SkyCoord(dec=(theta2-np.pi/2)*unit.rad,ra = phi2*unit.rad,distance=1*unit.m,frame='gcrs')\nsens_sat = SkyCoord(np.transpose(sensfilethetaphi)[1]*unit.rad,(np.transpose(sensfilethetaphi)[0]-np.pi/2)*unit.rad,frame='gcrs')\nnearest1 = astropy.coordinates.match_coordinates_sky(sens_source1, sens_sat, nthneighbor=1, storekdtree='kdtree_sky')[0]\n#nearest2 = astropy.coordinates.match_coordinates_sky(sens_source2, sens_sat, nthneighbor=1, storekdtree='kdtree_sky')[0]"

In [44]:
matchcoord = SkyCoord(phideg1*unit.degree,(thetadeg1-90)*unit.degree)
catalogcoord = SkyCoord(np.rad2deg(np.transpose(sensfilethetaphi)[1])*unit.degree,(np.rad2deg(np.transpose(sensfilethetaphi)[0]-np.pi/2))*unit.degree)
        

In [62]:
index = astropy.coordinates.match_coordinates_sky(matchcoord, catalogcoord, nthneighbor=1, storekdtree='kdtree_sky')[0]

In [72]:
#index[2],np.deg2rad(phideg1[2]),np.deg2rad((thetadeg1)[2]),(np.transpose(sensfilethetaphi)[1])[830]

In [58]:
sensitivitydata1 = []
sensitivitydata2 = []
sensfilethetaphi = []
for i in range(len(sensfile)):
    sensfilethetaphi.append((sensfile[i][0], sensfile[i][1]))
    

In [67]:
for i in range(len(theta)):
    sensitivitydata1.append(sensfile[index[i]][3])


In [69]:
for k in range(len(sensitivitydata1)):
    if np.isnan(sensitivitydata1[k])==True:
        sensitivitydata1[k] = 0
        
sensdatabackup1 = sensitivitydata1